# Préparation de la base de données
Il faut définir un jeu de données qui permettra d'entraîner le modèle. Il est possible d'utiliser un jeu de données public sur internet ou de le créer soi-même en prenant des images du ou des objet(s) en question.

Il faut ensuite labelliser les données manuellement sur toutes les images, c'est-à-dire délimiter les zones dans lesquels les objets sont situés sur les images. Il est possible de le faire via le logiciel gratuit : https://github.com/tzutalin/labelImg. Les étapes d'installation sont décrites dans le README.rst. Les étapes pour la labellisation également (Partie "Usage"->"Steps (YOLO)"). 

Après labellisation, il devrait y avoir des fichiers txt qui portent le même nom que les images et qui contiennent les coordonnées du ou des rectangles de délimitation contenant les objets. Il devrait également y avoir un fichier classes.txt, qui contient le nom des classes (objets à identifier).

# Entrainement d'un modèle avec YOLOv4 et Darknet
Description : guide pour entraîner un modèle YOLOv4 personnalisé qui permet de détecter et localiser un ou plusieurs objets sur une image ou une video.

**Il est conseillé d'utiliser ce notebook sur Google Colab car l'installation des fichiers open source que l'on va utiliser (Darknet) est complexe sur le pc et l'entraînement peut être bien plus long.**

**Si vous êtes sur Google Colab, veuillez aller dans le menu "Exécution" -> "Modifier le type d'execution", Sélectionner "GPU" et enregistrer.**

In [ ]:
# Verification de l'activation de NVIDIA GPU
!nvidia-smi

**Connexion et autorisation d'utiliser les dossiers de Google Drive avec Google Colab**

Le dossier de travail par défaut sur Google Colab peut être accédé en appuyant sur "Fichier" dans la colonne d'iconnes à droite de l'écran. Le problème de ce dossier est qu'il est volatile, c'est-à-dire que les modifications dans ce dossier seront perdus dès la fermeture du Notebook. Nous vous conseillons donc d'utiliser un dossier sur votre espace Drive en tant que dossier de travail.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow # colab does not support cv2.imshow()
from google.colab import drive
drive.mount('/content/drive')

**Ouverture du dossier de travail**

Le dossier de travail utilisé sur ce Notebook sera intitulé "YOLOv4" en assumant l'avoir créé manuellement dans votre espace Google Drive auparavant. Vous pouvez modifier le dossier de travail si vous le souhaitez.

Dans ce dossier, veuillez créer:

*   un dossier [doc_dataset], et copiez-y vos images, fichier classes et les fichiers contenant les coordonnées des annotations réalisées précédemment.
*   un dossier [doc_data] qui contiendra les fichiers de configurations, poids, et classes préentrainées (expliqué plus tard, pour l'instant, le dossier reste vide).


In [ ]:
%cd "/content/drive/MyDrive/YOLOv4"

**Récupération des fichiers Darknet**

Darknet est un réseau de neurone open-source qui est connu pour son architecture de détection d'objets en temps-réel. Il permet d'entraîner des modèles de détection d'objets personnalisés.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Veuillez ouvrir le fichier Makefile dans le dossier darknet téléchargé (vous pouvez l'ouvrir directement en double-cliquant dessus dans le volet à droite) et changez les valeurs suivantes :
```
GPU=1
CUDNN=1
OPENCV=1
```

Après avoir enregistré les modifications, veuillez exécuter les commandes suivantes :

In [ ]:
%cd "/content/drive/MyDrive/YOLOv4/darknet"
!chmod +rwx ./*
!make

**Découpage de la base de données**

Le code qui suit permet de découper la base de données en une base d'entrainement (80%) et une base de test (20%) et créera deux fichiers : objet_training.txt et objet_test.txt

In [ ]:
from os import listdir
from os.path import isfile, join
from google.colab import drive

mydirectory = "/content/drive/My Drive/YOLOv4/darknet/doc_dataset/doc_images"
allfiles = [f for f in listdir(mydirectory) if isfile(join(mydirectory, f))]
 
rootdir = "doc_dataset/doc_images/"
counter = 1
images_files = []
 
# put all the images files into a list
for file in allfiles:
  if (file[len(file)-3:len(file)] != "txt"):
    images_files.append(rootdir +  file)
 
# Set the size of the test (20%) & train data (80%)
training_limit = int(len(images_files) * 80 / 100)
test_limit = len(images_files) - training_limit
 
# Build the two lists
images_training_files = []
images_test_files = []
for file in images_files:
  if (counter <= training_limit):
    images_training_files.append(file)
  else:
    images_test_files.append( file)
  counter = counter + 1
 
# Put the two list into two files
with open('/content/drive/My Drive/YOLOv4/darknet/doc_dataset/objet_test.txt', 'w') as f:
    for item in images_test_files:
        f.write("%s\n" % item)
 
with open('/content/drive/My Drive/YOLOv4/darknet/doc_dataset/objet_training.txt', 'w') as f:
    for item in images_training_files:
        f.write("%s\n" % item)

# Création des fichiers de configuration d’entraînement

**Téléchargement des fichiers de configurations :**

Il vous faudra télécharger le fichier de poids suivant : https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
sur lequel l'entraînement se basera (transfer learning).

Il faudra également un fichier cfg personnalisé qui correspond au fichier de configuration du réseau de neurones. Les fichiers de configurations sont composés de deux parties :
*   Une section [net] définissant la configuration du réseau de neurones dans sa généralité
*   Une grosse section [layers] définissant les différentes couches du réseau de neurones

Dans la section [net], nous pouvons trouver des paramètres tels que:
*   **batch** : nombre d'échantillons (images, lettres, ...) pris en compte pour une itération
*   **subdivisions** : nombre de mini-batch dans un batch, taille d'un mini-batch = batch/subdivisions, le GPU traite les mini-batchs en même temps, et le poids est mis à jour après chaque batch (1 itération traite un seul batch d'images)
*   **width** : taille par défaut dans le réseau (largeur), chaque image est redimensionné à la taille par défaut pendant l'entraînement et la détection
*   **height** : taille par défaut dans le réseau (hauteur), chaque image est redimensionné à la taille par défaut pendant l'entraînement et la détection
*   **channels** : taille par défaut dans le réseau (canal), chaque image est redimensionné à la taille par défaut pendant l'entraînement et la détection
*   **max_batches**  : l'entraînement sera effectué pour *max_batches* itérations

Pour plus de détails, veuillez vous référer au lien suivant :
https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-%5Bnet%5D-section

La section [layers] définit chaque couche du réseau de neurones. Les paramètres de chaque couche sont décrits dans le lien suivant : https://github.com/AlexeyAB/darknet/wiki/CFG-Parameters-in-the-different-layers

Il existe déjà des fichiers de configurations dans le dossier .cfg. Vous pouvez copier le fichier *yolov4.cfg*. Il vous faudra tout-de-même modifier quelques éléments dans ce fichier car le nombre de classes pris en compte dans le fichier par défaut est de 80. Veuillez ouvrir le fichier et faire les modifications suivantes :
*   Ligne 3 : subdivisions=12
*   Ligne 7 : width=416
*   Ligne 8 : height=416
*   Ligne 19 : max_batches = 2000
*   Ligne 21 : steps=1600,1800
*   Ligne 961,1049,1137 : filters=18 # (classes+5)*3
*   Ligne 968,1056,1144 : classes=1

Pour des modèle de plus petite taille, vous pouvez utiliser le fichier de configuration *yolov4-tiny.cfg*. Nous avons également pu trouver plusieurs fichiers de configuration qui permettent d'entraîner des modèles de très petite taille sur le Github suivant:
https://github.com/dog-qiuqiu/MobileNet-Yolo

Sur ce repository Github, vous pouvez trouver un fichier de configuration *yoloface-50k.cfg* et *yoloface-500k-v2.cfg* que vous trouverez dans leur dossier respectif. Le fichier *yoloface-50k.cfg* permet d'entraîner un modèle de 50kB et le fichier *yoloface-500k-v2.cfg* permet d'entraîner un modèle de 500kB. Si vous souhaitez utiliser un de ces deux fichiers, veuillez vous assurer de le modifier afin que le fichier puisse correspondre a votre application, notamment le nombre de classes dans la couche [yolo] du réseau de neurones. Le fichier .cfg que nous avons utilisé pour entraîner notre modèle est disponible sur : https://github.com/ValFlashIC/Projet5A/blob/main/yolokeyboard-50k.cfg

**Avant d’entraîner le modèle :**

Veuillez créer un fichier objet.names dans ./darknet/doc_data et ajouter dedans la/les classe(s) d'objets a détecter.
Veuillez créer un autre fichier objet.data qui référence en quelque sorte les autres fichiers de configuration du réseau.
Le fichier objet.data respecte ces entrées :

```
classes = 1
valid = doc_dataset/objet_test.txt
train = doc_dataset/objet_training.txt
names = doc_dataset/objet.names
backup = backup
```

Explications :

*   classes : nombre précisant le nombre de classes du modèle
*   valid : chemin du fichier référençant les images de test
*   train : chemin du fichier référençant les images d’entrainement
*   names : fichier contenant la liste des noms de classes
*   backup : non-utilisé ici

# Entraînement du modèle

A ce stade, vous devez avoir créé une base de données et configuré les fichiers d'entraînement. Veuillez exécuter la cellule de code suivante afin de vous trouver dans le bon dossier.

In [ ]:
%cd "/content/drive/MyDrive/YOLOv4/darknet"
!chmod +rwx ./*

**Exécutez la cellule suivante afin de lancer l'entraînement du modèle.** 

Dans la commande de la cellule suivante, "*doc_dataset/objet.data*" correspond au fichier data comprenant le noms des classes d'objets, "*cfg/yoloface-50k.cfg*" correspond au fichier de configuration du réseau de neurones et "*doc_dataset/yolov4.conv.137*" correspond au fichier de poids original sur lequel le transfer learning est effectué.

Cet entrainement peut durer des heures selon le nombre de classes et la configuration du réseau de neurones.

In [ ]:
!./darknet detector train doc_dataset/objet.data cfg/yoloface-50k.cfg doc_dataset/yolov4.conv.137 -dont_show -map

Pour suivre l'avancée de l'entraînement, vous verrez de nombreux messages apparaître comme ceci precisant le numéro de l'itération actuelle (1906) et le taux de perte (avg loss):

```
 (next mAP calculation at 2000 iterations) 
 1906: 0.756686, 1.466187 avg loss, 0.000013 rate, 22.043027 seconds, 114360 images, 1.476337 hours left
Loaded: 11.882894 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.361245), count: 1, class_loss = 0.201240, iou_loss = 0.875642, total_loss = 1.076881 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.766196), count: 13, class_loss = 1.758310, iou_loss = 3.690968, total_loss = 5.449277 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.822551), count: 40, class_loss = 1.684688, iou_loss = 3.024157, total_loss = 4.708845 
 total_bbox = 3317, rewritten_bbox = 0.000000 % 
```
Vous pouvez également suivre l'entraînement en regardant le graphe enregistré dans le dossier darknet appelé: chart_yoloface-50k.png. Ce graphe illustre la courbe d'apprentissage avec en rouge les valeurs de mAP (mean Average Precision expliqué dans la notice) et en bleu la Loss (l'erreur a minimiser).

Une erreur peut apparaître pendant l'entraînement s'il est trop long : il se peut que Google collab stoppe l'entrainement en indiquant que le temps d'utilisation du GPU est fini et que vous devez attendre. Darknet effectue des sauvegardes régulièrement ce qui permet de reprendre l'entraînement s'il le faut. 

Pour modifier ces paramètres de backup automatique, il suffit de modifier le fichier .cfg qui a été créé précédemment :

```
burn_in=1000 (permet de créer une copie du fichier de poids après la millième itération)
max_batches = 2000 (l'entraînement s’arrète après 2000 itérations)
```

En cas d’erreur, il suffit de lancer la ligne de commande suivante pour que darknet reprenne a partir de sa derniere sauvegarde.

In [ ]:
!./darknet detector train doc_dataset/objet.data cfg/yoloface-50k.cfg backup/yoloface-50k_last.weights -dont_show -map

# Test du modèle

Apres avoir terminé l'entraînement du modèle, vous pouvez utiliser une image au hasard afin de le tester.

Il se peut que l'image ne s'affiche pas dans Google collab. Vous pouvez verifier le résultat avec l'image "*prediction.jpg*" créée après exécution des commandes suivantes.

In [ ]:
# Commande pour aller dans le bon dossier
%cd "/content/drive/MyDrive/YOLOv4/darknet"
!chmod +rwx ./*

Les paramètres de la commande suivante sont :
*   "*doc_dataset/objet.data*" qui correspond au fichier .data avec le nom des classes
*   "*cfg/yoloface-50k.cfg*" le fichier de configuration du réseau de neurones
*   "*backup/yoloface-50k_best.weights*" le modèle sous format .weights
*   "*doc_dataset/doc_images/claviertest.jpg*" l'image sur laquelle tester le modèle

In [ ]:
!./darknet detector test doc_dataset2/document.data "/content/drive/MyDrive/YOLOv4/darknet/cfg/yoloface-50k.cfg"  "/content/drive/MyDrive/YOLOv4/darknet/backup/yoloface-50k_best.weights"  "/content/drive/MyDrive/YOLOv4/darknet/doc_dataset/doc_images/claviertest.jpg"

Vous pouvez également tester le modèle sur une video avec la commande suivante. Les paramètres importants sont :

*   "*doc_dataset/objet.data*" qui correspond au fichier .data avec le nom des classes
*   "*cfg/yoloface-50k.cfg*" le fichier de configuration du réseau de neurones
*   "*backup/yoloface-50k_best.weights*" le modèle sous format .weights
*   "*video/test.mp4*" la video sur laquelle tester le modèle
*   "*resultat.avi*" la video de sortie avec le résultat


In [ ]:
!./darknet detector demo doc_dataset2/document.data cfg/yolokeyboard-50k.cfg backup/yolokeyboard-50k_best.weights video/test3.mp4 -dont_show {} -i 0 -out_filename resultat3_yolokeyboard.avi
!du -h resultat3_yolokeyboard.avi